In [1]:
# INSTALLING REQUIREMENTS
%pip install pandas transformers datasets torch huggingface_hub accelerate -U

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# INCLUDING THE LIBRARIES
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [3]:
# INCLUDING THE DATA FROM HUGGING FACE
df = pd.read_parquet("hf://datasets/DELith/github-issues/pandas-issues.parquet")

In [4]:
df.head()

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,updated_at,closed_at,author_association,active_lock_reason,body,reactions,timeline_url,performed_via_github_app,draft,pull_request
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/1,337721,MDU6SXNzdWUzMzc3MjE=,1,Enable element-wise comparison operations in D...,...,2014-02-11 20:40:01+00:00,2011-02-19 23:13:48+00:00,MEMBER,None,"re: pystatsmodels e-mail\n\nhi everyone,\n\nju...","{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/pandas-dev/pandas...,NaN,NaN,None
1,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/2,337726,MDU6SXNzdWUzMzc3MjY=,2,reindex_like function,...,2014-09-15 17:20:09+00:00,2010-12-17 02:57:33+00:00,MEMBER,None,"In Series, DataMatrix, WidePanel, etc.\n","{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/pandas-dev/pandas...,NaN,NaN,None
2,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/3,337728,MDU6SXNzdWUzMzc3Mjg=,3,Binary operations on int DataMatrix,...,2014-09-15 17:20:09+00:00,2011-01-01 23:50:12+00:00,MEMBER,None,"Reported by kwgoodman, Jun 12, 2010\nWhen addi...","{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/pandas-dev/pandas...,NaN,NaN,None
3,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/4,337730,MDU6SXNzdWUzMzc3MzA=,4,Plot keyword arguments are unused in DataFrame...,...,2014-02-11 20:40:01+00:00,2010-12-11 06:14:32+00:00,MEMBER,None,"Reported by twhitcomb, Jul 27, 2010\nWhat step...","{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/pandas-dev/pandas...,NaN,NaN,None
4,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/issues/5,337736,MDU6SXNzdWUzMzc3MzY=,5,Python 2.7 testing,...,2014-02-11 20:40:01+00:00,2010-12-17 02:46:34+00:00,MEMBER,None,Test pandas on Python 2.7\n,"{'+1': 0, '-1': 0, 'confused': 0, 'eyes': 0, '...",https://api.github.com/repos/pandas-dev/pandas...,NaN,NaN,None


In [5]:
# FEATURE CONVERTING
def feature_converter(data):
    label_mapping = {
        # New Feature
        'enhancement': 'New_feature',
        'ideas': 'New_feature',
        'api design': 'New_feature',
        # Improvement
        'refactor': 'Improvement',
        'build': 'Improvement',
        'performance': 'Improvement',
        'docs': 'Improvement',
        'unicode': 'Improvement',
        'visualization': 'Improvement',
        # Bug
        'bug': 'Bug',
        'error reporting': 'Bug',
        'regression': 'Bug',
        'duplicate report': 'Bug',
    }
    data['class'] = data['label'].apply(lambda x: label_mapping.get(x, 'Task'))
    return data[['title', 'body', 'class']]

In [6]:

def data_parser(data):
    data_filtered = data[data['labels'].apply(lambda x: len(x) > 0)]
    data_filtered['label'] = data_filtered['labels'].apply(lambda x: x[0]['name'].lower() if len(x) > 0 else None)
    parsed_data = data_filtered[['label', 'title', 'body']]
    final_data = feature_converter(parsed_data)
    return final_data

data = data_parser(df)

/tmp/ipykernel_124890/3736565802.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['label'] = data_filtered['labels'].apply(lambda x: x[0]['name'].lower() if len(x) > 0 else None)
/tmp/ipykernel_124890/3712085686.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['class'] = data['label'].apply(lambda x: label_mapping.get(x, 'Task'))


In [7]:
data.head()

,title,body,class
7,np.fix doesn't work,"```\nIn [28]: np.fix(Series([1,2,3], range(3))...",Bug
12,Add PyTable Paths to HDFStore,"Hello,\nI not sure if this was the best way to...",New_feature
16,Better support for mixed-type data,Across DataFrame and Wide/LongPanel classes\n,New_feature
17,weights option may not be working in pandas.st...,Need to investigate (user notified)\n,Bug
38,Adding variable description in DataFrame,"Hi,\n\nI think it would be great if DataFrame ...",New_feature


In [8]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

# DOWNLOADING MODEL
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=4)
def preprocess_function(examples):
    texts = [
        (title if title is not None else "") + " " + (body if body is not None else "")
        for title, body in zip(examples["title"], examples["body"])
    ]
    return tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=512
    )

# CONVERTING THE DATA TO DATASET
dataset = Dataset.from_pandas(data)

# TOKENIZE THE DATA
tokenized_dataset = dataset.map(preprocess_function, batched=True)

/home/uxino/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/38927 [00:00<?, ? examples/s]

In [ ]:
# INITIALING CLASS NAME
label_mapping = {'Bug': 0, 'Improvement': 1, 'New_feature': 2, 'Task': 3}
data['label'] = data['class'].map(label_mapping)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

data['title'] = data['title'].fillna('').astype(str)
data['body'] = data['body'].fillna('').astype(str)

# TOKENIZATION
encodings = tokenizer(
    list(data['title']),
    list(data['body']),
    truncation=True, 
    padding=True, 
    max_length=512, 
    return_tensors='pt'
)

train_dataset = CustomDataset(encodings, list(data['label']))

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    learning_rate=6e-5,                  
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()